In [1]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd

import psycopg2

from pymongo import MongoClient
from pprint import pprint

from neo4j import GraphDatabase, RoutingControl

import faker

import time
import random

from collections import defaultdict

In [3]:
def generate_data(N:int) -> pd.DataFrame:
    """Generate five dataframes with N rows"""

    fake = faker.Faker()
    users = []
    cities = []
    resumes = []
    companies = []
    resume_company = []

    print("Generating users ...")
    for id in range(10):
        users.append({
            "id": id,
            "login": fake.user_name(),
            "password": fake.password(),
        })
    
    print("Generating cities ...")
    cities.append({
        "id": 0,
        "name": "Rivne"
    })
    for id in range(1, int(N/10)):
        cities.append({
            "id": id,
            "name": fake.city()
        })

    print("Generating resumes ...")
    for id in range(N):
        resumes.append({
            "id": id,
            "name": fake.name(),
            "email": fake.email(),
            "phone": fake.phone_number(),
            "city_id": random.choice(cities)['id'],
            "education": fake.random_element(elements=("High School", "Bachelor", "Master", "PhD")),
            "hobby": fake.random_element(elements=("sport", "reading", "music", "drawing")),
            "experience": fake.job()
        })
    
    print("Generating companies ...")
    for id in range(N):
        companies.append({
            "id": id,
            "name": fake.company()
        })
    
    print("Generating connection between companies and resumes ...")
    for id in range(10*N):
        r_c = {
            "id": id,
            "resume_id": random.choice(resumes)['id'],
            "company_id": random.choice(companies)['id']
        }
        if r_c not in resume_company:
            resume_company.append(r_c)

    users, cities, resumes, companies, resume_company = pd.DataFrame(users), pd.DataFrame(cities), pd.DataFrame(resumes), pd.DataFrame(companies), pd.DataFrame(resume_company)
    return {"users": users, "cities": cities, "resumes": resumes, "companies": companies, "resume_company": resume_company}

# Generating data
start = time.time()
tables = generate_data(5000)
end = time.time()
print(f"Time: {end - start}")

Generating users ...
Generating cities ...
Generating resumes ...
Generating companies ...
Generating connection between companies and resumes ...
Time: 76.29869651794434


In [46]:
def create_table_query(name:str, df:pd.DataFrame) -> str:
    query = f"CREATE TABLE IF NOT EXISTS {name} (\n"
    for column in df.columns:
        if column == "id":
            query += f"\t{column} serial PRIMARY KEY,\n"
        elif "id" in column:
            query += f"\t{column} serial REFERENCES {column[:-4] + 'ies' if column[:-3][-1] == 'y' else column[:-3]+'s'},\n"
        else:
            query += f"\t{column} VARCHAR,\n"
    query = query[:-2] + "\n);"
    return query

def insert_query(table:str, df:pd.DataFrame) -> str:
    query = f"INSERT INTO {table} ("
    for column in df.columns:
        query+=column + ", "
    query = query[:-2] + ") VALUES\n"
    for id in df.index:
        query += "("
        for column in df.columns:
            if "id" in column:
                query += str(df[column][id]) + ", "
            else:
                query += "'" + df[column][id].replace("'", "") + "', "
        query = query[:-2] + "),\n"
    query = query[:-2]+";"
    return query

# Creating and filling tables in Postgresql
start = time.time()
conn = psycopg2.connect("dbname=hr_system user=admin password=admin port=5432")
with conn.cursor() as cur:
    for table in tables:
        cur.execute(create_table_query(table, tables[table]))
        cur.execute(insert_query(table, tables[table]))
end = time.time()
print(f"Time: {end - start}")
    

Time: 18.812623500823975


In [47]:
postgres_queries = [
    "SELECT * FROM resumes",
    "SELECT hobby FROM resumes",
    "SELECT DISTINCT c.name FROM resumes AS r LEFT JOIN cities AS c ON r.city_id = c.id",
    "SELECT r.hobby FROM resumes AS r LEFT JOIN cities AS c ON r.city_id = c.id WHERE c.name = 'Rivne'",
    "SELECT c.name, string_agg(r.name, ', ') FROM resumes AS r RIGHT JOIN resume_company AS rc ON r.id = rc.resume_id LEFT JOIN companies AS c ON rc.company_id = c.id GROUP BY c.name"
]

# Executing queries
for query in postgres_queries:
    start = time.time()
    with conn.cursor() as cur:
        cur.execute(query)
        dt = cur.fetchall()
        print(query)
        print(f"Fetched rows: {len(dt)}")
        print(f"Five first rows:")
        for row in dt[:5]:
            print(row)
    end = time.time()
    print(f"Time: {end - start}")
    print()

SELECT * FROM resumes
Fetched rows: 5000
Five first rows:
(0, 'Patricia Lopez', 'alexandria20@example.org', '5519708541', 186, 'Master', 'sport', 'Risk analyst')
(1, 'Brittany Byrd', 'hjones@example.net', '001-837-709-0309x10457', 366, 'High School', 'music', 'Surveyor, insurance')
(2, 'Ryan Weaver', 'angelabray@example.org', '495-292-4482x959', 366, 'Master', 'drawing', 'Publishing copy')
(3, 'Ashley Lopez', 'ycooper@example.org', '918-861-1115x34719', 375, 'Master', 'reading', 'Training and development officer')
(4, 'Desiree Kelly', 'mary36@example.com', '(620)924-6102x90415', 53, 'High School', 'sport', 'Therapist, art')
Time: 0.06300187110900879

SELECT hobby FROM resumes
Fetched rows: 5000
Five first rows:
('sport',)
('music',)
('drawing',)
('reading',)
('sport',)
Time: 0.008999109268188477

SELECT DISTINCT c.name FROM resumes AS r LEFT JOIN cities AS c ON r.city_id = c.id
Fetched rows: 487
Five first rows:
('Lambertborough',)
('West Jenniferside',)
('Nelsonberg',)
('New Christoph

In [48]:
# Disconecting from Postgresql
conn.close()

In [49]:
# Creating and filling tables in Mongodb

def to_mongo(tables:dict):
    new_resumes = []
    for i in tables['resumes'].index:
        new_resumes.append({
            'name': tables['resumes']['name'][i],
            'email': tables['resumes']['email'][i],
            'phone': tables['resumes']['phone'][i],
            'city': tables['cities'][tables['cities']['id'] == tables['resumes']['city_id'][i]]['name'].iloc[0],
            'education': tables['resumes']['education'][i],
            'hobby': tables['resumes']['hobby'][i],
            'companies': [comp for comp in tables['companies'][
                tables['companies']['id'].isin(
                    tables['resume_company'][tables['resume_company']['resume_id'] == tables['resumes']['id'][i]]['company_id']
                    )]['name']]
        })
    return [("users", tables['users']), ("resumes", pd.DataFrame(new_resumes))]

start = time.time()
client = MongoClient("localhost", 27017, username="admin", password="admin")
for name, table in to_mongo(tables):
    m_table = client.hr_system[name]
    m_table.insert_many(table.to_dict("records"))
end = time.time()
print(f"Time: {end - start}")

Time: 24.645453929901123


In [50]:
# Executing queries
start = time.time()
table = [row for row in client.hr_system['resumes'].find()]
print(f"Fetched rows: {len(table)}")
print("Five first rows:")
for row in table[:5]:
    print(row)
end = time.time()
print(f"Time: {end-start}\n")

start = time.time()
table = [row for row in client.hr_system['resumes'].find({}, {"hobby": 1})]
print(f"Fetched rows: {len(table)}")
print("Five first rows:")
for row in table[:5]:
    print(row)
end = time.time()
print(f"Time: {end-start}\n")

start = time.time()
table = [row for row in client.hr_system['resumes'].find({}, {"city": 1}).distinct("city")]
print(f"Fetched rows: {len(table)}")
print("Five first rows:")
for row in table[:5]:
    print(row)
end = time.time()
print(f"Time: {end-start}\n")

start = time.time()
table = [row for row in client.hr_system['resumes'].find({"city": "Rivne"}, {"hobby": 1})]
print(f"Fetched rows: {len(table)}")
print("Five first rows:")
for row in table[:5]:
    print(row)
end = time.time()
print(f"Time: {end-start}\n")

start = time.time()
table = [row for row in client.hr_system['resumes'].find()]

comp_names = defaultdict(list)
for row in table:
    for company in row['companies']:
        comp_names[company].append(row['name'])

print(f"Fetched rows: {len(comp_names)}")
print("Five first rows:")
for i, row in enumerate(comp_names):
    print(f"{row}, {comp_names[row]}")
    if i >= 5:
        break

end = time.time()
print(f"Time: {end-start}\n")


Fetched rows: 5000
Five first rows:
{'_id': ObjectId('6534fd66bcb9137ec280b393'), 'name': 'Patricia Lopez', 'email': 'alexandria20@example.org', 'phone': '5519708541', 'city': 'Port Charlesburgh', 'education': 'Master', 'hobby': 'sport', 'companies': ['Smith, Tucker and Santos', 'Jones, Mitchell and Krause', 'Harmon-Lewis', 'Mcdonald-Rivera', 'Bailey, Thompson and Boyer']}
{'_id': ObjectId('6534fd66bcb9137ec280b394'), 'name': 'Brittany Byrd', 'email': 'hjones@example.net', 'phone': '001-837-709-0309x10457', 'city': 'Andrewland', 'education': 'High School', 'hobby': 'music', 'companies': ['Williams Group', 'Scott-Sutton', 'Hall LLC', 'Solis LLC', 'Carroll-Luna', 'Hayden Inc', 'Henderson-Roth', 'Parker, Stephenson and Ramos', 'Vang, Hayes and Wang', 'Calhoun PLC', 'Smith-Gray', 'Roach-Garcia', 'White Ltd', 'Jackson Inc', 'Smith, Johnston and Martinez', 'Ryan Ltd']}
{'_id': ObjectId('6534fd66bcb9137ec280b395'), 'name': 'Ryan Weaver', 'email': 'angelabray@example.org', 'phone': '495-292-44

In [51]:
# Disconecting from Mongodb
client.close()

In [4]:
# Creating and filling tables in Neo4j
start = time.time()
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "strongpassword"))

print("Creating users ...")
for user in tables['users'].to_dict("records"):
    driver.execute_query(
        "CREATE (user:User {id: $id, login: $login, password: $password})",
        id = user['id'], login = user['login'], password = user['password'], database_="neo4j",
    )
driver.execute_query(
    "CREATE INDEX user_id FOR (u:User) ON (u.id)",
    database_="neo4j",
)

print("Creating cities ...")
for city in tables['cities'].to_dict("records"):
    driver.execute_query(
        "CREATE (:City {id: $id, name : $name})",
        id = city['id'], name = city['name'], database_="neo4j",
    )
driver.execute_query(
    "CREATE INDEX city_id FOR (c:City) ON (c.id)",
    database_="neo4j",
)

print("Creating resumes ...")
for resume in tables['resumes'].to_dict("records"):
    driver.execute_query(
        "MATCH (city:City {id: $city_id})"
        "CREATE (:Resume {id: $id, name: $name, email: $email, phone: $phone, education: $education, hobby: $hobby, experience: $experience}) -[:LIVE_IN]-> (city)",
        city_id = resume['city_id'],
        id = resume['id'],
        name = resume['name'],
        email = resume['email'],
        phone = resume['phone'],
        education = resume['education'],
        hobby = resume['hobby'],
        experience = resume['experience'],
        database_= "neo4j",
    )
driver.execute_query(
    "CREATE INDEX resume_id FOR (r:Resume) ON (r.id)",
    database_="neo4j",
)

print("Creating companies ...")
for company in tables['companies'].to_dict("records"):
    driver.execute_query(
        "CREATE (:Company {id: $id, name: $name})",
        id = company['id'], name = company['name'], database_= "neo4j",
    )
driver.execute_query(
    "CREATE INDEX company_id FOR (c:Company) ON (c.id)",
    database_="neo4j",
)

print("Connecting resumes and companies ...")
for r_c in tables['resume_company'].to_dict("records"):
    driver.execute_query(
        "MATCH (r:Resume {id: $resume_id})"
        "MATCH (c:Company {id: $company_id})"
        "CREATE (r) -[:WORKED_IN]-> (c)",
        resume_id = r_c['resume_id'], company_id = r_c['company_id'], database_= "neo4j",
    )

end = time.time()
print(f"Time: {end - start}")

Time: 1236.6341898441315


In [44]:
# Executing queries

start = time.time()
nodes, _, _ = driver.execute_query(
    "MATCH (r:Resume) RETURN r",
    database_="neo4j", routing_=RoutingControl.READ,
)

print(f"Fetched nodes: {len(nodes)}")
print("Five first nodes:")
for resume in nodes[:5]:
    print(resume['r'].__dict__['_properties'])
end = time.time()
print(f"Time: {end - start}")

start = time.time()
nodes, _, _ = driver.execute_query(
    "MATCH (r:Resume) RETURN r.hobby",
    database_="neo4j", routing_=RoutingControl.READ,
)

print(f"Fetched nodes: {len(nodes)}")
print("Five first nodes:")
for resume in nodes[:5]:
    print(resume['r.hobby'])
end = time.time()
print(f"Time: {end - start}")

start = time.time()
nodes, _, _ = driver.execute_query(
    "MATCH (c:City) <-[:LIVE_IN]- () RETURN DISTINCT c.name",
    database_="neo4j", routing_=RoutingControl.READ,
)

print(f"Fetched nodes: {len(nodes)}")
print("Five first nodes:")
for resume in nodes[:5]:
    print(resume['c.name'])
end = time.time()
print(f"Time: {end - start}")

start = time.time()
nodes, _, _ = driver.execute_query(
    "MATCH (r:Resume) -[:LIVE_IN]-> (c:City {name: $city_name}) RETURN r.hobby",
    city_name = "Rivne", database_="neo4j", routing_=RoutingControl.READ,
)

print(f"Fetched nodes: {len(nodes)}")
print("Five first nodes:")
for resume in nodes[:5]:
    print(resume['r.hobby'])
end = time.time()
print(f"Time: {end - start}")

start = time.time()
nodes, _, _ = driver.execute_query(
    "MATCH (c:Company) <-[:WORKED_IN]- (r:Resume) RETURN c.name, collect(r.name) as names",
    database_="neo4j", routing_=RoutingControl.READ,
)

print(f"Fetched nodes: {len(nodes)}")
print("Five first nodes:")
for resume in nodes[:5]:
    print(f"{resume['c.name']}: {resume['names']}")
end = time.time()
print(f"Time: {end - start}")

Fetched nodes: 5000
Five first nodes:
{'education': 'Master', 'phone': '5519708541', 'name': 'Patricia Lopez', 'id': 0, 'experience': 'Risk analyst', 'email': 'alexandria20@example.org', 'hobby': 'sport'}
{'education': 'High School', 'phone': '001-837-709-0309x10457', 'name': 'Brittany Byrd', 'id': 1, 'experience': 'Surveyor, insurance', 'email': 'hjones@example.net', 'hobby': 'music'}
{'education': 'Master', 'phone': '495-292-4482x959', 'name': 'Ryan Weaver', 'id': 2, 'experience': 'Publishing copy', 'email': 'angelabray@example.org', 'hobby': 'drawing'}
{'education': 'Master', 'phone': '918-861-1115x34719', 'name': 'Ashley Lopez', 'id': 3, 'experience': 'Training and development officer', 'email': 'ycooper@example.org', 'hobby': 'reading'}
{'education': 'High School', 'phone': '(620)924-6102x90415', 'name': 'Desiree Kelly', 'id': 4, 'experience': 'Therapist, art', 'email': 'mary36@example.com', 'hobby': 'sport'}
Time: 1.740241527557373
Fetched nodes: 5000
Five first nodes:
sport
musi

In [45]:
# Disconecting from Neo4j
driver.close()